# Notebook 3.

In this notebook we will use the power of transfer learning to achieve some pretty cool results. 

Step 1. Extract images from videos we took at the start.

In [1]:
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

In [2]:
import cv2
import sys
import os
import imutils
import numpy as np
import random
import keras

Using TensorFlow backend.


In [3]:
labels_dict = {'IMG_2221.MOV':0,
               'IMG_2222.MOV':1
               }

In [4]:
face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")

for key in labels_dict.keys():
    
    vidcap = cv2.VideoCapture('videos/{}'.format(key))
    success, image = vidcap.read()
    count = 0
    success = True
        
    while success:
        image = imutils.rotate(image, 270)
        faces = face_cascade.detectMultiScale(image, 1.1, 3, minSize=(100, 100))
        
        
        i=0
        for (x, y, w, h) in faces:
            
            r = max(w, h) / 2
            centerx = x + w / 2
            centery = y + h / 2
            nx = int(centerx - r)
            ny = int(centery - r)
            nr = int(r * 2)

            faceimg = image[ny:ny+nr, nx:nx+nr]
            lastimg = cv2.resize(faceimg, (192, 192))

            i += 1
            cv2.imwrite("images/train/{0}-{1}.jpg".format(str(labels_dict[key]),count), lastimg)
    
        success, image = vidcap.read()      
        count += 1
    

Load data

In [5]:
DIR = 'images/train/'

def load_training_data():
    train_data = []
    for img in os.listdir(DIR):
        label = img[0]
        if label != '.':
            path = os.path.join(DIR, img)
            img = cv2.imread(path)
            train_data.append([np.array(img), label])

    return np.array(train_data)

In [6]:
data = load_training_data()

In [7]:
random.shuffle(data)

In [8]:
labels = [int(el[1]) for el in data]

In [9]:
images = np.array([el[0] for el in data])

In [10]:
one_hot_labels = keras.utils.to_categorical(labels, num_classes=len(list(set(labels))))

In [11]:
one_hot_labels[:2]

array([[0., 1.],
       [0., 1.]], dtype=float32)

In [12]:
# class_labels = np.array(list(labels_dict.values()))
# class_labels

In [13]:
from keras import Sequential
from keras.layers import Flatten, Dense, Dropout, Dense

pretrained_model = keras.applications.MobileNetV2(input_shape=[*[192,192], 3], include_top=False)
pretrained_model.trainable = False

transfer = keras.Sequential([
                            pretrained_model,
                            keras.layers.Flatten(),
                            keras.layers.Dense(2, activation='softmax')
                            ])

In [14]:
transfer.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [15]:
transfer.fit(images, one_hot_labels, epochs=10, batch_size=16)

Epoch 1/10
148/148 [==============================] - 6s 41ms/step - loss: 0.1941 - accuracy: 0.9189
Epoch 2/10
148/148 [==============================] - 5s 31ms/step - loss: 1.6109e-09 - accuracy: 1.0000
Epoch 3/10
148/148 [==============================] - 4s 30ms/step - loss: 4.0273e-09 - accuracy: 1.0000
Epoch 4/10
148/148 [==============================] - 4s 30ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 5/10
148/148 [==============================] - 4s 30ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 6/10
148/148 [==============================] - 4s 30ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 7/10
148/148 [==============================] - 4s 30ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 8/10
148/148 [==============================] - 4s 30ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 9/10
148/148 [==============================] - 4s 30ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 10/10
148/148 [==============================] - 4s 3

In [17]:
import matplotlib.pyplot as plt

face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")

image = cv2.imread('/images.test.jpg')

# image = imutils.rotate(image, 270)
faces = face_cascade.detectMultiScale(image, 1.1, 3, minSize=(100, 100))

i=0
for (x, y, w, h) in faces[:1]:
 
    r = max(w, h) / 2
    centerx = x + w / 2
    centery = y + h / 2
    nx = int(centerx - r)
    ny = int(centery - r)
    nr = int(r * 2)

    faceimg = image[ny:ny+nr, nx:nx+nr]
    lastimg = cv2.resize(faceimg, (192, 192))
    
    lastimg_reshaped = np.expand_dims(lastimg, axis=0) 
    plt.title(transfer.predict_classes(lastimg_reshaped)[0])
    cv2.imshow(str(), lastimg)
    cv2.waitKey(0)
    
    

